<a href="https://colab.research.google.com/github/NerdyWithZainab/OCR/blob/main/OCR_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("preatcher/standard-ocr-dataset")

print("Path to dataset files:", path)

100%|██████████| 46.2M/46.2M [00:03<00:00, 15.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/preatcher/standard-ocr-dataset/versions/2


In [ ]:
!pip install kaggle


In [ ]:
!kaggle datasets download -d preatcher/standard-ocr-dataset

Dataset URL: https://www.kaggle.com/datasets/preatcher/standard-ocr-dataset
License(s): CC0-1.0
100% 46.0M/46.2M [00:03<00:00, 24.6MB/s]
100% 46.2M/46.2M [00:03<00:00, 15.5MB/s]


In [ ]:
# Unzip the dataset
!unzip /content/standard-ocr-dataset.zip -d /content/standard-ocr-dataset

Streaming output truncated to the last 5000 lines.
  inflating: /content/standard-ocr-dataset/data2/training_data/R/50083.png  
  inflating: /content/standard-ocr-dataset/data2/training_data/R/50119.png  
  inflating: /content/standard-ocr-dataset/data2/training_data/R/50155.png  
  inflating: /content/standard-ocr-dataset/data2/training_data/R/50191.png  
  inflating: /content/standard-ocr-dataset/data2/training_data/R/50227.png  
  inflating: /content/standard-ocr-dataset/data2/training_data/R/50263.png  
  inflating: /content/standard-ocr-dataset/data2/training_data/R/50299.png  
  inflating: /content/standard-ocr-dataset/data2/training_data/R/50335.png  
  inflating: /content/standard-ocr-dataset/data2/training_data/R/50371.png  
  inflating: /content/standard-ocr-dataset/data2/training_data/R/50407.png  
  inflating: /content/standard-ocr-dataset/data2/training_data/R/50443.png  
  inflating: /content/standard-ocr-dataset/data2/training_data/R/50479.png  
  inflating: /content/sta

In [ ]:
# Import the necessary libraries to read the images and datasets
import torch
import torch.nn as nn
import os
import random
from PIL import Image
from torchvision import transforms
import cv2
import numpy as np
from torch.utils.data import DataLoader, random_split
from torchvision import datasets

In [ ]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Defining image transformations (resize and normalize)
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((32, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load the train and test datasets
train_dataset = datasets.ImageFolder(root='/content/standard-ocr-dataset/data', transform=transform)

Using device: cuda


### Training and splitting datasets

In [ ]:
# Split dataset (e.g., 80% train, 20% test)
train_size = int(0.8 * len(train_dataset))
test_size = len(train_dataset) - train_size

train_data, test_data = random_split(train_dataset, [train_size, test_size])

# DataLoaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [ ]:
import torch
import torch.nn as nn

# Simple CNN-based OCR model
class OCRModel(nn.Module):
    def __init__(self, num_classes):
        super(OCRModel, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.fc = nn.Linear(128 * 8 * 32, num_classes)  # Adjust based on input size

    def forward(self, x):
        x = self.cnn(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc(x)
        return x

# Initialize the model
model = OCRModel(num_classes=len(train_dataset.classes)).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(100):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if epoch % 10 == 0:
      print(f"Epoch: {epoch} | Loss: {loss:.5f}%")


Epoch: 0 | Loss: 0.54353%


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-88-ced71cedff7e>", line 33, in <cell line: 0>
    for images, labels in train_loader:
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 701, in __next__
    data = self._next_data()
           ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 756, in _next_data
    index = self._next_index()  # may raise StopIteration
            ^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 691, in _next_index
    return next(self._sampler_iter)  # may raise StopIteration
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/sampler.py", line None, in __iter__
KeyboardInterrupt

During handling 

In [ ]:
import os
import random
from PIL import Image
import torch
from torchvision import transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # Move the model to the device
print(f"Using device:{device}")
# Character set (should match the training character set)
character_set = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
idx_to_char = {i: c for i, c in enumerate(character_set)}

model = model.to(device)  # Move the model to the device
# Load your trained CRNN model
model.eval()

# Ensure the output layer has the correct number of classes (if needed)
num_classes = len(character_set)  # Should be 36
if hasattr(model, 'fc') and isinstance(model.fc, nn.Linear):
    model.fc = nn.Linear(model.fc.in_features, num_classes).to(device)

# Step 1: Randomly select an image from the dataset folder
dataset_folder = "/content/standard-ocr-dataset/data/testing_data/"  # Replace with your dataset folder
all_images = []

# Walk through the folder to find all images
for root, _, files in os.walk(dataset_folder):
    for file in files:
        if file.endswith((".png", ".jpg", ".jpeg")):  # Include common image formats
            all_images.append(os.path.join(root, file))

# Randomly select an image
selected_image_path = random.choice(all_images)
print(f"Selected image: {selected_image_path}")

# Ensure the output layer matches the character set length
num_classes = len(character_set)  # Should be 36

# Step 2: Load the selected image
image = Image.open(selected_image_path).convert("L")  # Convert to grayscale

# Step 3: Apply the same transformations used during training
transform = transforms.Compose([
    transforms.Resize((32, 128)),  # Ensure the input size matches your model
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Ensure normalization matches training
])
input_image = transform(image).unsqueeze(0).to(device)  # Add batch dimension and move to device

# Step 4: Perform inference
for name, param in model.named_parameters():
    if param.device != device:
        param.data = param.data.to(device)
try:
    with torch.no_grad():
        output = model(input_image)
    print(f"Output shape: {output.shape}")
except Exception as e:
    print(f"Error during inference: {e}")

output = output.squeeze(0)
# Check the shape of the output
print("Output shape:", output.shape)

# Step 5: If output has the batch dimension, remove it
if output.dim() == 2:
    output = output.squeeze(0)  # Remove batch dimension

# Step 6: Inspect the output (raw logits before argmax)
print("Raw model output (logits):", output)
print("Max value and index:", output.max().item(), output.argmax().item())
# Step 7: Convert output logits to predicted character indices
# Here, we apply argmax along dimension 0 since output is 1D after squeeze
predicted_index = torch.argmax(output).item()


# Step 8: Decode the sequence (remove repeated characters)
# For sequence prediction, you'd need a loop to decode each timestep
# Here, we assume a single character prediction
decoded_text = idx_to_char.get(predicted_index, '')  # Get character, handle invalid index

# Print the predicted text
print(f"Predicted text: {decoded_text}")

Using device:cuda
Selected image: /content/standard-ocr-dataset/data/testing_data/G/28578.png
Output shape: torch.Size([1, 36])
Output shape: torch.Size([36])
Raw model output (logits): tensor([-0.0219, -0.0165,  0.0167,  0.0191,  0.0086,  0.0099,  0.0038, -0.0142,
         0.0753,  0.0326, -0.0177, -0.0300, -0.0101, -0.0076, -0.0287, -0.0330,
        -0.0153,  0.0049,  0.0255, -0.0281,  0.0267, -0.0163, -0.0054, -0.0410,
        -0.0120, -0.0184, -0.0133,  0.0004,  0.0086, -0.0283, -0.0334, -0.0092,
        -0.0010,  0.0106, -0.0082, -0.0029], device='cuda:0')
Max value and index: 0.07533742487430573 8
Predicted text: I
